In [145]:
##### # Imports

# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
from random import randint

# No warnings about setting value on copy of slice
pd.options.mode.chained_assignment = None

# Display up to 60 columns of a dataframe
pd.set_option('display.max_columns', 60)

# Matplotlib visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Set default font size
plt.rcParams['font.size'] = 24

# Internal ipython tool for setting figure size
from IPython.core.pylabtools import figsize

# Seaborn for visualization
import seaborn as sns
sns.set(font_scale = 2)

# Splitting data into training and testing
from sklearn.model_selection import train_test_split

#Imputing missing values and scaling values
from sklearn.preprocessing import Imputer, MinMaxScaler

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

#Machine Learning Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
#from xgboost import XGBRegressor


# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [34]:
raw_df = pd.read_csv('processed/df_sorted.csv', parse_dates=["issue_d", "last_pymnt_d"]) #read training data

In [116]:
raw_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,13000,13000,11525.000000,36 months,0.1062,423.28,1,6 years,RENT,86000.0,Not Verified,0,1.0,debt_consolidation,12.35,0,0,10,0,9871,0.456,23,0,0,0,15161.065730,13440.86,13000.0,2161.07,0.0,0.0,0.0,34,2907.14,0,1,0,0,0,0.0,0
1,7000,7000,6785.947887,36 months,0.0751,217.77,0,4 years,RENT,80000.0,Source Verified,0,1.0,major_purchase,14.88,0,0,10,1,8433,0.413,17,0,0,0,7838.664792,7582.93,7000.0,838.66,0.0,0.0,0.0,36,455.06,0,1,0,0,0,1.0,0
2,12000,12000,11196.564730,36 months,0.1025,388.62,1,1 year,RENT,48645.0,Not Verified,0,1.0,debt_consolidation,13.64,0,0,10,0,13039,0.310,17,0,0,0,13922.041770,12966.90,12000.0,1922.04,0.0,0.0,0.0,33,2701.47,0,1,0,0,0,0.0,0
3,22750,22750,21931.621590,36 months,0.1136,748.73,1,5 years,RENT,58600.0,Not Verified,0,1.0,debt_consolidation,24.66,0,1,8,0,1449,0.296,37,0,0,0,25428.895730,24460.37,22750.0,2678.90,0.0,0.0,0.0,13,14957.35,0,1,0,0,0,0.0,0
4,15500,15500,15500.000000,36 months,0.1645,548.36,4,2 years,MORTGAGE,415000.0,Not Verified,0,1.0,credit_card,0.68,0,2,11,0,11886,0.563,31,0,0,0,19418.699580,19418.70,15500.0,3918.70,0.0,0.0,0.0,25,5188.83,0,1,0,0,0,0.0,0


In [53]:
raw_df['issue_d'] = raw_df['issue_d'].astype('category').cat.codes
#raw_df['revol_util'] = raw_df['revol_util'].str.rstrip('%').astype('float') / 100.0

In [56]:
raw_df['last_pymnt_d'] = raw_df['last_pymnt_d'].astype('category').cat.codes

In [60]:
raw_df.initial_list_status = 0

In [77]:
category_vars_int = ['issue_d','last_pymnt_d','initial_list_status','grade']

In [76]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42317 entries, 0 to 42316
Data columns (total 41 columns):
loan_amnt                     42317 non-null int64
funded_amnt                   42317 non-null int64
funded_amnt_inv               42317 non-null float64
term                          42317 non-null object
int_rate                      42317 non-null float64
installment                   42317 non-null float64
grade                         42317 non-null int8
emp_length                    42317 non-null object
home_ownership                42317 non-null object
annual_inc                    42317 non-null float64
verification_status           42317 non-null object
issue_d                       42317 non-null int8
loan_status                   42317 non-null object
purpose                       42317 non-null object
dti                           42317 non-null float64
delinq_2yrs                   42317 non-null int64
inq_last_6mths                42317 non-null int64
open_acc  

In [78]:
category = ['purpose', 'verification_status', 'home_ownership', 'emp_length', 'term']

In [85]:
labels = raw_df['loan_status']

In [90]:
dic = {
    "Fully Paid" :1,
    "Charged Off":0
}

In [93]:
raw_df['loan_status'] = raw_df['loan_status'].map(dic)

In [95]:
raw_df.tail()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
42312,2500,2500,2475.000000,36 months,0.0740,77.65,0,3 years,OTHER,186000.0,Not Verified,52,1.0,major_purchase,4.06,0,1,15,0,14063,0.502,33,0,0,0,2746.653811,2719.19,2500.0,246.65,0.0,0.0,0.0,13,1121.49,0,1,0,0,0,0.0,0
42313,8000,8000,8000.000000,36 months,0.1148,263.75,1,10+ years,RENT,36000.0,Verified,52,1.0,debt_consolidation,2.90,0,1,7,0,2980,0.317,10,0,0,0,9494.897134,9494.90,8000.0,1494.90,0.0,0.0,0.0,34,280.33,0,1,0,0,0,0.0,0
42314,18500,18500,18175.000000,36 months,0.1287,622.22,2,3 years,RENT,62004.0,Verified,52,1.0,debt_consolidation,11.84,0,0,14,0,9530,0.460,40,0,0,0,21102.058460,20731.35,18500.0,2602.06,0.0,0.0,0.0,16,11775.21,0,1,0,0,0,0.0,0
42315,12000,12000,11750.000000,36 months,0.1218,399.60,1,1 year,OWN,41004.0,Verified,52,1.0,home_improvement,5.24,0,2,6,0,10117,0.363,9,0,0,0,12241.862120,11986.83,12000.0,241.86,0.0,0.0,0.0,109,11843.99,0,1,0,0,0,0.0,0
42316,9925,9925,9814.100582,36 months,0.1148,327.22,1,4 years,MORTGAGE,45000.0,Verified,52,1.0,other,12.67,1,3,12,0,696,0.027,31,0,0,0,11779.570270,11637.87,9925.0,1854.57,0.0,0.0,0.0,35,346.22,0,1,0,0,0,0.0,0


In [96]:
features = raw_df.drop(columns="loan_status") 

In [97]:
labels = raw_df['loan_status']

In [104]:
assert features.shape[0] == labels.shape[0]

In [107]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42317 entries, 0 to 42316
Data columns (total 40 columns):
loan_amnt                     42317 non-null int64
funded_amnt                   42317 non-null int64
funded_amnt_inv               42317 non-null float64
term                          42317 non-null object
int_rate                      42317 non-null float64
installment                   42317 non-null float64
grade                         42317 non-null int8
emp_length                    42317 non-null object
home_ownership                42317 non-null object
annual_inc                    42317 non-null float64
verification_status           42317 non-null object
issue_d                       42317 non-null int8
purpose                       42317 non-null object
dti                           42317 non-null float64
delinq_2yrs                   42317 non-null int64
inq_last_6mths                42317 non-null int64
open_acc                      42317 non-null int64
pub_rec    

In [112]:
features_ohe = pd.get_dummies(features)

In [196]:
features_ohe = features_ohe.drop(columns="total_rec_prncp")

In [197]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [198]:
X_train , X_valid = split_vals(features_ohe, 20000)
y_train , y_valid = split_vals(labels, 20000)
y_valid = np.ravel(y_valid)
y_train = np.ravel(y_train)
y_valid.shape

(22317,)

In [199]:
X_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,last_pymnt_d,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,...,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,13000,13000,11525.000000,0.1062,423.28,1,86000.0,0,12.35,0,0,10,0,9871,0.456,23,0,0,0,15161.065730,13440.86,2161.07,0.0,34,2907.14,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,7000,7000,6785.947887,0.0751,217.77,0,80000.0,0,14.88,0,0,10,1,8433,0.413,17,0,0,0,7838.664792,7582.93,838.66,0.0,36,455.06,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,12000,12000,11196.564730,0.1025,388.62,1,48645.0,0,13.64,0,0,10,0,13039,0.310,17,0,0,0,13922.041770,12966.90,1922.04,0.0,33,2701.47,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,22750,22750,21931.621590,0.1136,748.73,1,58600.0,0,24.66,0,1,8,0,1449,0.296,37,0,0,0,25428.895730,24460.37,2678.90,0.0,13,14957.35,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,15500,15500,15500.000000,0.1645,548.36,4,415000.0,0,0.68,0,2,11,0,11886,0.563,31,0,0,0,19418.699580,19418.70,3918.70,0.0,25,5188.83,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [200]:
cols = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'annual_inc', 'dti', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc','out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_int',
       'total_rec_late_fee','last_pymnt_amnt', 'collections_12_mths_ex_med']

In [205]:
## Default model
m=RandomForestClassifier(n_estimators=200)


%time m.fit(X_train, y_train)

Wall time: 7.51 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [206]:
print("Accuracy:",metrics.accuracy_score(y_valid, m.predict(X_valid)))

Accuracy: 0.9363265671909307


In [207]:
feat_imp = pd.DataFrame({"Columns": X_train.columns, 
                         "importance": m.feature_importances_})

In [208]:
feat_imp.sort_values("importance",ascending=False)

,Columns,importance
20,total_pymnt_inv,0.146815
19,total_pymnt,0.145039
24,last_pymnt_amnt,0.128092
23,last_pymnt_d,0.091678
1,funded_amnt,0.059825
2,funded_amnt_inv,0.059024
4,installment,0.051722
0,loan_amnt,0.044432
21,total_rec_int,0.041127
3,int_rate,0.026800


In [210]:
X_train.to_csv('processed/training_features.csv')
X_valid.to_csv('processed/testing_features.csv')
pd.DataFrame(y_train).to_csv('processed/training_labels.csv')
pd.DataFrame(y_valid).to_csv('processed/testing_labels.csv')
